<a href="https://colab.research.google.com/github/Madhura0802/Matrix_Mul/blob/main/Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
code = """
#include <stdio.h>
#include <stdlib.h>
#define N 1024
#define BLOCK_SIZE 16
__global__ void matrixMul(int *a, int *b, int *c, int width) {
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
int sum = 0;
for (int i = 0; i < width; i++) {
sum += a[row * width + i] * b[i * width + col];
}
c[row * width + col] = sum;
}
int main() {
int *a, *b, *c;
int *d_a, *d_b, *d_c;
int size = N * N * sizeof(int);
// Allocate memory on host
a = (int*)malloc(size);
b = (int*)malloc(size);
c = (int*)malloc(size);
// Initialize matrices
for (int i = 0; i < N; i++) {
for (int j = 0; j < N; j++) {
a[i * N + j] = i + j;
b[i * N + j] = i - j;
} }
// Allocate memory on device
cudaMalloc(&d_a, size);
cudaMalloc(&d_b, size);
cudaMalloc(&d_c, size);
// Copy data from host to device
cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);
// Launch kernel with 2D grid and 2D block
dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
dim3 dimGrid((N + dimBlock.x - 1) / dimBlock.x, (N + dimBlock.y - 1) / dimBlock.y);
matrixMul<<<dimGrid, dimBlock>>>(d_a, d_b, d_c, N);
// Copy result from device to host
cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);
// Print first and last elements of result
printf("c[0][0] = %d, c[%d][%d] = %d", c[0], N-1, N-1, c[(N-1) * N + (N-1)]);
// Free memory
free(a);
free(b);
free(c);
cudaFree(d_a);
cudaFree(d_b);
cudaFree(d_c);
return 0;
}
"""

In [3]:
text_file = open("Matrix.cu","w")
text_file.write(code)
text_file.close()

In [4]:
!nvcc Matrix.cu

In [5]:
!./a.out

c[0][0] = 357389824, c[1023][1023] = -714255872

In [6]:
!nvprof ./a.out

==805== NVPROF is profiling process 805, command: ./a.out
==805== Profiling application: ./a.out
c[0][0] = 357389824, c[1023][1023] = -714255872==805== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   69.86%  9.1119ms         1  9.1119ms  9.1119ms  9.1119ms  matrixMul(int*, int*, int*, int)
                   17.34%  2.2618ms         1  2.2618ms  2.2618ms  2.2618ms  [CUDA memcpy DtoH]
                   12.80%  1.6694ms         2  834.72us  809.25us  860.19us  [CUDA memcpy HtoD]
      API calls:   90.03%  159.75ms         3  53.251ms  102.37us  159.55ms  cudaMalloc
                    8.78%  15.589ms         3  5.1963ms  1.0342ms  13.515ms  cudaMemcpy
                    0.69%  1.2220ms         1  1.2220ms  1.2220ms  1.2220ms  cuDeviceGetPCIBusId
                    0.35%  628.94us         3  209.65us  194.18us  222.26us  cudaFree
                    0.10%  178.95us       101  1.7710us     181ns  72.262us  cuDeviceGe